In [2]:
function Cost(Ktu, dtKtu, ddtKtu, Kuu, I, θ::Union{PoissonProcessParams, AbstractArray}, KernList::AbstractArray;
              η_RKHS=1e1, η_Cm = 1e1)
    
    C, α, u, N, M, D, J = name_params(θ)
    
    cost = 0;
    num_spikes = 0
    
    for n = 1:N
        cur_spike = 0;
        for m = 1:M
            num_spikes += I[n][m]
            for i = 1:I[n][m]
                tmp_cost_deriv = 0
                tmp_cost_deriv2 = 0
                cur_spike += 1
                cur_inducing = 0
                for d = 1:D
                    for j = 1:J[d]
                        cur_inducing += 1
                        tmp_cost_deriv += C[m,d]*α[n,d][j]*dtKtu[n][cur_spike, cur_inducing]
                        tmp_cost_deriv2 += C[m,d]*α[n,d][j]*ddtKtu[n][cur_spike, cur_inducing]
                    end
                end
                
                cost += 0.5*tmp_cost_deriv.^2 + tmp_cost_deriv2
            end
        end
    end
    

    # Computing the RKHS norm
    rkhs_norm = 0
    for n = 1:N
        for d = 1:D
            d_inds = (d>1?cumsum(J)[d-1]+1:1):cumsum(J)[d];
            rkhs_norm += α[n,d]'*Kuu[n][d_inds,d_inds]*α[n,d]
        end
    end
    
    # Computing the C-norm
    C_2norm = 0
    C_infnorm = 0
    for m = 1:M
        C_2norm += norm(C[m,:])
        C_infnorm += norm(C[m, :], Inf)
    end
    
    return cost/num_spikes + η_RKHS*rkhs_norm + η_Cm * C_2norm
end



function Cost(data::AbstractArray, θ::Union{PoissonProcessParams, AbstractArray}, KernList::AbstractArray;
              η_RKHS=1e1, η_Cm = 1e1)
    
    # Compute the kernels
    Ktu, dtKtu, ddtKtu, Kuu, I = build_kernels(data, θ, KernList)
    
    # Pass them on onto the kernely version
    return Cost(Ktu, dtKtu, ddtKtu, Kuu, I, θ, KernList;
              η_RKHS=η_RKHS, η_Cm = η_Cm)
end

LoadError: LoadError: UndefVarError: PoissonProcessParams not defined
while loading In[2], in expression starting on line 1